# Hyperparameter Tuning using HyperDrive

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Environment, ScriptRunConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import pandas as pd
import os
import joblib

In [2]:
ws = Workspace.from_config()
ws.write_config(path='.azureml')
experiment_name = 'capstone-gridstability'

exp = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

#run = exp.start_logging()

print(ws)

Workspace name: quick-starts-ws-183469
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-183469
Workspace.create(name='quick-starts-ws-183469', subscription_id='d7f39349-a66b-446e-aba6-0053c2cf1c11', resource_group='aml-quickstarts-183469')


In [3]:
cpu_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Cluster already exists. Use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)

Cluster already exists. Use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
env = Environment.get(workspace = ws, name = "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

In [5]:
"""
if "training" not in os.listdir():
    os.mkdir("./training")
"""

'\nif "training" not in os.listdir():\n    os.mkdir("./training")\n'

In [6]:
src = ScriptRunConfig(source_directory = ".",
                      script = "train.py",
                      compute_target = "my-cluster",
                      environment = env
                     )

## Dataset

In [7]:
os.listdir()

['.amlignore',
 '.amlignore.amltmp',
 '.azureml',
 '.ipynb_aml_checkpoints',
 'data',
 'hyperparameter_tuning.ipynb',
 'hyperparameter_tuning.ipynb.amltmp',
 'outputs',
 'train.py',
 'train.py.amltmp']

In [8]:
ds = pd.read_csv("data/gridstability.csv")

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [9]:
# Specify parameter sampler
ps = RandomParameterSampling({"--C":choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
                             "--max_iter":choice(50,100,200,300)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=1)

In [10]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             policy=policy,
                             max_total_runs=16,
                             max_concurrent_runs=4)

## Run Details

In [11]:
# Submit the hyperdrive run to the experiment and show run details with the widget.
hd_run = exp.submit(hd_config)

# The widget RunDetails can be used to monitor the run
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [12]:
hd_run.wait_for_completion(show_output=True)

RunId: HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e
Web View: https://ml.azure.com/runs/HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-183469/workspaces/quick-starts-ws-183469&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-02-10T17:51:05.886852][API][INFO]Experiment created<END>\n""<START>[2022-02-10T17:51:06.762689][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-02-10T17:51:07.488199][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e
Web View: https://ml.azure.com/runs/HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-183469/workspaces/quick-starts-ws-183469&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e',
 'target': 'my-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-10T17:51:05.641519Z',
 'endTimeUtc': '2022-02-10T17:59:17.664638Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '9cf8c5fc-33d6-4c9e-a2d7-fa88a38f116a',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1064-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.37.0',
  'space_size': '44',
  'score': '0.8175',
  'best_child_run_id': 'HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e_7',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg183469.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e/azureml-logs/hyperdriv

In [13]:
print(hd_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

[{'run_id': 'HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e_7', 'hyperparameters': '{"--C": 50, "--max_iter": 50}', 'best_primary_metric': 0.8175, 'status': 'Completed'}, {'run_id': 'HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e_8', 'hyperparameters': '{"--C": 100, "--max_iter": 50}', 'best_primary_metric': 0.817, 'status': 'Completed'}, {'run_id': 'HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e_15', 'hyperparameters': '{"--C": 1, "--max_iter": 200}', 'best_primary_metric': 0.8165, 'status': 'Completed'}, {'run_id': 'HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e_3', 'hyperparameters': '{"--C": 200, "--max_iter": 50}', 'best_primary_metric': 0.8165, 'status': 'Completed'}, {'run_id': 'HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e_14', 'hyperparameters': '{"--C": 10, "--max_iter": 100}', 'best_primary_metric': 0.816, 'status': 'Completed'}, {'run_id': 'HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e_12', 'hyperparameters': '{"--C": 500, "--max_iter": 200}', 'best_primary_metric': 0.816, 'status': 'Completed'}, {'run_id': 'HD_7

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
print(hd_run.get_children_sorted_by_primary_metric(top=1, reverse=False, discard_no_metric=False))

[{'run_id': 'HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e_7', 'hyperparameters': '{"--C": 50, "--max_iter": 50}', 'best_primary_metric': 0.8175, 'status': 'Completed'}]


In [15]:
best_run = hd_run.get_best_run_by_primary_metric()
best_run.get_file_names()

['system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'user_logs/std_log.txt']

In [16]:
best_run.get_metrics()

{'Regularization Strength:': 50.0, 'Max iterations:': 50, 'Accuracy': 0.8175}

In [17]:
best_run.get_details()

{'runId': 'HD_7fdaf4d0-9647-4a35-9148-a03ca2ac844e_7',
 'target': 'my-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-10T17:55:09.996647Z',
 'endTimeUtc': '2022-02-10T17:55:13.607673Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '9cf8c5fc-33d6-4c9e-a2d7-fa88a38f116a',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '50', '--max_iter', '50'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'my-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'en

In [23]:
best_run.register_model(model_name = 'best_model_hd')

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path best_model_hd in the set of files uploaded to the run: ['system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path best_model_hd in the set of files uploaded to the run: ['system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']\n                See https://aka.ms/run-logging for more details.